# Working notebook to get the Reference Point Reversal script working without Matlab

## Setup

In [9]:
import pydicom
import os
import sys
import time
import copy
import inspect

In [15]:
sourceFile = os.path.join(os.getcwd(), 'RP.dcm')
testFile = os.path.join(os.getcwd(), 'RP-Rev-Matlab.dcm')
saveFile = sourceFile.replace('.dcm', '-Reversed.dcm')
filename = os.path.basename(sourceFile)
path = os.path.dirname(sourceFile)
referencePlan = pydicom.dcmread(sourceFile)
testPlan = pydicom.dcmread(testFile)
# Create a copy of the original file
flippedPlan = copy.deepcopy(referencePlan)
# Modify the copy name to indicate it's been reversed
flippedPlan.RTPlanLabel = referencePlan.RTPlanLabel + "-Reversed"
flippedPlan.SOPInstanceUID = pydicom.uid.generate_uid()
# Plan name safety, if the plan name is too long it will be cut
if (len(flippedPlan.RTPlanLabel) > 16):
    flippedPlan.RTPlanLabel = referencePlan.RTPlanLabel[0:11] + "-Rev"



## Testing

In [ ]:
print("Modifying the file...")
print(sourceFile)
print(filename)
print(path)
print(referencePlan.RTPlanLabel)
print(flippedPlan.RTPlanLabel)

In [ ]:
print(referencePlan)

## Algorithm

In [ ]:
for i in range(len(referencePlan.BeamSequence)):
    CPcount = len(referencePlan.BeamSequence[i].ControlPointSequence)
    CPList = list(referencePlan.BeamSequence[i].ControlPointSequence)
    ControlPoints = referencePlan.BeamSequence[i].ControlPointSequence

    for j in range(CPcount):
        if j == 0:
            Col = CPList[j].BeamLimitingDeviceAngle
            if Col > 180:
                Col = Col - 180
            else:
                Col = Col + 180
            flippedPlan.BeamSequence[i].ControlPointSequence[j].BeamLimitingDeviceAngle = Col
        Dir = CPList[j].GantryRotationDirection
        if Dir == 'CW':
            flippedPlan.BeamSequence[i].ControlPointSequence[j].GantryRotationDirection = 'CC'
        elif Dir == 'CC':
            flippedPlan.BeamSequence[i].ControlPointSequence[j].GantryRotationDirection = 'CW'
        elif Dir == 'NONE':
            print('Done')
            print(i)
        else:
            raise ValueError('No Gantry')
        swapAngle = CPList[CPcount - j - 1].GantryAngle
        flippedPlan.BeamSequence[i].ControlPointSequence[j].GantryAngle = swapAngle
flippedPlan.save_as(saveFile)


## Testing

In [ ]:
testBeams = len(testPlan.BeamSequence)
flippedBeams = len(flippedPlan.BeamSequence)
comparisons = 0
if testBeams != flippedBeams:
    #raise ValueError('The number of beams in the test plan does not match the flipped plan')
    print('The number of beams in the test plan does not match the flipped plan')
for i in range (testBeams):
    testCPcount = len(testPlan.BeamSequence[i].ControlPointSequence)
    testControlPoints = testPlan.BeamSequence[i].ControlPointSequence
    flippedCPcount = len(flippedPlan.BeamSequence[i].ControlPointSequence)
    flippedControlPoints = flippedPlan.BeamSequence[i].ControlPointSequence
    if testCPcount != flippedCPcount:
        #raise ValueError('The number of control points in the test plan does not match the flipped plan')
        print('The number of control points in the test plan does not match the flipped plan')
    else :
        comparisons = comparisons + 1
    for j in range(testCPcount):
        testGantry = testControlPoints[j].GantryAngle 
        flippedGantry = flippedControlPoints[j].GantryAngle
        if testGantry != flippedGantry:
            #raise ValueError('The gantry angle of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
            print('The gantry angle of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
        else:
            comparisons = comparisons + 1
        testGantryRotationDirection = testControlPoints[j].GantryRotationDirection
        flippedGantryRotationDirection = flippedControlPoints[j].GantryRotationDirection
        if testGantryRotationDirection != flippedGantryRotationDirection:
            #raise ValueError('The gantry rotation direction of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
            print('The gantry rotation direction of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
        else:
            comparisons = comparisons + 1
        for k in range (len(testControlPoints[j].BeamLimitingDevicePositionSequence)):
            testLeaf = testControlPoints[j].BeamLimitingDevicePositionSequence[k].LeafJawPositions
            flippedLeaf = flippedControlPoints[j].BeamLimitingDevicePositionSequence[k].LeafJawPositions
            for l in range (len(testLeaf)):
                if testLeaf[l] != flippedLeaf[l]:
                    #raise ValueError('The leaf jaw positions of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
                    print ('The leaf jaw positions of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
                else:
                    comparisons = comparisons + 1
            if testControlPoints[j].BeamLimitingDevicePositionSequence[k].LeafJawPositions != flippedControlPoints[j].BeamLimitingDevicePositionSequence[k].LeafJawPositions:
                #raise ValueError('The leaf jaw positions of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
                print('The leaf jaw positions of control point ' + str(j) + ' in beam ' + str(i) + ' does not match the flipped plan')
            else:
                comparisons = comparisons + 1
print('Comparisons: ' + str(comparisons))
            
